In [ ]:
import numpy as np
import time
import math as mt
import pickle  
from itertools import combinations ,permutations  

In [ ]:
times=np.array(range(0,1000001, 100), dtype='float')/1e12 #declare times array 0 to 1micro second with 100 ps

In [ ]:
molecules={
    'Naphtalene'        : 200.9e-9,
    'Anthracene'        : 5.8e-9, 
    'Benzopyrene'       : 38.6e-9,
    'Pyrene'            : 516.2e-9,
    'Chrysene'          : 57.8e-9,
    'Benzofluoranthene' : 8.9e-9,
} #molecules dictionary

In [ ]:
lifetimes=np.array(list(molecules.values())) #get lifetimes of 

In [ ]:
molecule_names=list(molecules.keys()) #get name of molecules

In [ ]:
def florescent(time, lifetime):
  return mt.exp(-(time/lifetime)) #calculate flouresence level

In [ ]:
fluoresence_levels=[] #get fluoresence level values of all molecules without A
for i in lifetimes:
  row=np.zeros(10000) #create 1 row per molecule to keep f.levels
  for j in range(10000):
    row[j]= florescent(times[j], i) #calc and append value
  fluoresence_levels.append(row) #append data of molecule
fluoresence_levels=np.array(fluoresence_levels)

In [ ]:
def show_errors(error_list, test, data_test, numbers): #show errors between mixture data and guessed data
  for i in range( numbers ):
    print(f'Guessed          {test[error_list[i]]}')
    print(f'Real data        {data_test[error_list[i]]}')

In [ ]:
def errors(test, data_test): #find number of errors and their indexes
  errors=0
  error_list= []
  for i in range(10000):
    if test_error(test[i], data_test[i]): #check value that guessed in data is error
      error_list.append(i) #collect index of error
  error_list=np.array(error_list)
  return error_list

In [ ]:
def test_error(value1, value2): #check value to determine it is error or not
  if abs(value1-value2) > 1e-10: #if abstract of two value is greater than 10^-10 then it is error
    return True
  return False

In [ ]:
def get_sum(a_vals, index_list): # get guessed data with the indexes of molecules and A values
  sum=np.zeros(10000)
  for i in range(len(index_list)):
    sum+=a_vals[i]*fluoresence_levels[index_list[i]]
  return sum

In [92]:
def A_list (size, n): #determine all the possible A values respect to the sum of A and number of molecules

  comb=np.ones(n*(size-n+1)) 
  temp=1

  for i in range(n, n*(size-n+1), n):
    comb[i:i+n]+=temp #prepare data to apply permutation function for taking into consederation same values
    temp+=1 

  a_list=[]
  for i in permutations(comb, n):
    if(sum(i)==size and i not in a_list): #find cases that have sum is equal A sum
      a_list.append(i)
  return np.array(a_list)

In [98]:
def mixture_composition( data_test, num_mixture ): #show result and return error with giving data and number of mixtures
  A_sum=int(data_test[0]) #get A sum from first value of mixture data (t=0 exp=1 and there will be only sum of A)
  less_errors=9999 #minimum error variable
  A_comb=A_list( A_sum , num_mixture ) #get combinations of A
  t_comb=combinations(range(0,6), num_mixture ) #get combination of molecules
  
  for t_vals in combinations(range(0,6), num_mixture ): #iterate all the combinations
    for A_vals in A_list( A_sum , num_mixture ):
      guessed_data= get_sum (A_vals, t_vals) #generate data
      num_of_errors = len( errors( guessed_data, data_test ) ) #get number of errors
      if num_of_errors < less_errors: #compare with lest error, if less than lest, lest will be that
        A_lessErrors=A_vals #keep A combination of less error 
        t_lessErrors=t_vals #keep molecule combination of less error
        less_errors = num_of_errors #keep less error value
        if (less_errors==0): #if it is zero break out of all loops
          break
    else:
      continue
    break

  for i in range(1, len(t_lessErrors)+1): #show molecules and its A values in composition
    print(f' {molecule_names[ t_lessErrors[i-1] ]} ( { lifetimes[ t_lessErrors[i-1] ] } )  A={A_lessErrors[i-1]}')
  return less_errors

# Mixture 1

In [ ]:
with open('mixture1.dat', 'rb') as FID:
  myUnpickler = pickle.Unpickler(FID)
  data=myUnpickler.load()

data1=np.array(data, dtype='float')

In [ ]:
print( f' Mixture 1 with { mixture_composition ( data1, 2 ) } errors ' )

 Naphtalene ( 2.009e-07 )  A=1.0
 Benzofluoranthene ( 8.9e-09 )  A=3.0
 Mixture 1 with 0 errors 


# Mixture 4

In [ ]:
with open('mixture4.dat', 'rb') as FID:
  myUnpickler = pickle.Unpickler(FID)
  data=myUnpickler.load()

data4=np.array(data, dtype='float')

In [ ]:
print( f' Mixture 4 with { mixture_composition ( data4, 3) } errors ' )

 Naphtalene ( 2.009e-07 )  A=1.0
 Benzopyrene ( 3.86e-08 )  A=2.0
 Benzofluoranthene ( 8.9e-09 )  A=3.0
 Mixture 4 with 0 errors 


# Mixture 2


In [ ]:
with open('mixture2.dat', 'rb') as FID:
  myUnpickler = pickle.Unpickler(FID)
  data=myUnpickler.load()

data2=np.array(data, dtype='float')

In [ ]:
print( f' Mixture 2 with { mixture_composition ( data2, 3 ) }  errors ' )

 Anthracene ( 5.8e-09 )  A=2.0
 Pyrene ( 5.162e-07 )  A=1.0
 Chrysene ( 5.78e-08 )  A=5.0
 Mixture 2 with 0  errors 


# Mixture 3


In [ ]:
with open('mixture3.dat', 'rb') as FID:
  myUnpickler = pickle.Unpickler(FID)
  data=myUnpickler.load()

data3=np.array(data.copy(), dtype='float')

In [ ]:
print( f' Mixture 3 with { mixture_composition ( data3, 2 ) }  errors ' )

 Naphtalene ( 2.009e-07 )  A=1.0
 Pyrene ( 5.162e-07 )  A=1.0
 Mixture 3 with 0  errors 


# Automaticly guessed how many molecules

In [99]:
def mixture_composition2 (data_test): #show result and return error with giving data
  A_sum=int(data_test[0]) #get A sum from first value of mixture data (t=0 exp=1 and there will be only sum of A)
  less_errors=9999 #minimum error variable

  for num_mixture in range(2,7):#iterate number of molecules
    A_comb=A_list( A_sum , num_mixture ) #get combinations of A
    t_comb=combinations(range(0,6), num_mixture ) #get combination of molecules

    for t_vals in combinations(range(0,6), num_mixture ): #iterate all the combinations
      for A_vals in A_list( A_sum , num_mixture ):
        guessed_data= get_sum (A_vals, t_vals) #generate data
        num_of_errors = len( errors( guessed_data, data_test ) ) #get number of errors
        if num_of_errors < less_errors: #compare with lest error, if less than lest, lest will be that
          A_lessErrors=A_vals #keep A combination of less error 
          t_lessErrors=t_vals #keep molecule combination of less error
          less_errors = num_of_errors #keep less error value
          if (less_errors==0): #if it is zero break out of all loops
            break
      else:
        continue
      break
    else:
      continue
    break
  for i in range(1, len(t_lessErrors)+1):
    print(f' {molecule_names[ t_lessErrors[i-1] ]} ( { lifetimes[ t_lessErrors[i-1] ] } )  A={A_lessErrors[i-1]}')
  return less_errors

In [101]:
print( f' Mixture 1 with { mixture_composition2( data1) }  errors ' )

 Naphtalene ( 2.009e-07 )  A=1.0
 Benzofluoranthene ( 8.9e-09 )  A=3.0
 Mixture 1 with 0  errors 


In [102]:
print( f' Mixture 2 with { mixture_composition2( data2) }  errors ' )

 Anthracene ( 5.8e-09 )  A=2.0
 Pyrene ( 5.162e-07 )  A=1.0
 Chrysene ( 5.78e-08 )  A=5.0
 Mixture 2 with 0  errors 


In [105]:
print( f' Mixture 3 with { mixture_composition2( data3) }  errors ' )

 Naphtalene ( 2.009e-07 )  A=1.0
 Pyrene ( 5.162e-07 )  A=1.0
 Mixture 3 with 0  errors 


In [104]:
print( f' Mixture 4 with { mixture_composition2( data4) }  errors ' )

 Naphtalene ( 2.009e-07 )  A=1.0
 Benzopyrene ( 3.86e-08 )  A=2.0
 Benzofluoranthene ( 8.9e-09 )  A=3.0
 Mixture 4 with 0  errors 
